In [10]:
import numpy as np
np.random.seed(42)
import matplotlib.pyplot as plt
import seaborn as sns
from subprocess import check_output
import os
import pandas as pd
%matplotlib inline

print("Checking files in data location ...")
drop_dir = '/Users/Gabriel/Dropbox/Research/ASPIDE/Datasets/ECI Chaos/Distributed Phase 1'
print(check_output(["ls", drop_dir]).decode("utf8"))
# Logs
anomaly_logs_dir = os.path.join(drop_dir, 'logs')
print("Anomaly log dir: {}".format(anomaly_logs_dir))
data_dir = drop_dir
print("Data dir: {}".format(data_dir))

processed_dir = os.path.join(data_dir, 'processed')
print("Processed dir: {}".format(processed_dir))
# Node ID list
# nodes = ['master', '1', '2', '3']
nodes = ['master']

Checking files in data location ...
audsome_single_anomaly_nooverlap.log
done
logs
misc
pr_data_audsome_anomaly_single_nooverlap.csv
processed

Anomaly log dir: /Users/Gabriel/Dropbox/Research/ASPIDE/Datasets/ECI Chaos/Distributed Phase 1/logs
Data dir: /Users/Gabriel/Dropbox/Research/ASPIDE/Datasets/ECI Chaos/Distributed Phase 1
Processed dir: /Users/Gabriel/Dropbox/Research/ASPIDE/Datasets/ECI Chaos/Distributed Phase 1/processed


In [11]:
# df_clean = pd.read_csv(os.path.join(data_dir,"pr_data_clean(65m).csv"))
df_anomaly = pd.read_csv(os.path.join(data_dir,"pr_data_audsome_anomaly_single_nooverlap.csv"))
# df_audsome = pd.read_csv(os.path.join(data_dir,"pr_data_audesome_clean(1h).csv"))
# df_audsome_anomaly = pd.read_csv(os.path.join(data_dir,"pr_data_audesome_anomalies(65m).csv"))
# print("Clean data shape: {}".format(df_clean.shape))
print("Anomaly data shape: {}".format(df_anomaly.shape))
# print("Clean AUDSOME data shape: {}".format(df_audsome.shape))
# print("Anomaly AUDSOME data shape: {}".format(df_audsome_anomaly.shape))

Anomaly data shape: (4800, 774)


In [12]:
# Drop first column
# df_clean.drop(["Unnamed: 0"], axis=1, inplace=True)
df_anomaly.drop(["Unnamed: 0"], axis=1, inplace=True)
# df_audsome.drop(["Unnamed: 0"], axis=1, inplace=True)
# df_audsome_anomaly.drop(["Unnamed: 0"], axis=1, inplace=True)

In [13]:
# For single node filtering
node_wild_card = "10.251.0.114:9100"
time = df_anomaly.time
#
filtr_list = []
mask = df_anomaly.columns.str.contains(node_wild_card)
filtr_list.extend(list(df_anomaly.loc[:,mask].columns.values))
df_anomaly = df_anomaly[filtr_list]
df_anomaly['time'] = time
df_anomaly

,node_arp_entries_10.251.0.114:9100,node_boot_time_seconds_10.251.0.114:9100,node_context_switches_total_10.251.0.114:9100,node_cpu_guest_seconds_total_10.251.0.114:9100,node_cpu_seconds_total_10.251.0.114:9100,node_disk_io_now_10.251.0.114:9100,node_disk_io_time_seconds_total_10.251.0.114:9100,node_disk_io_time_weighted_seconds_total_10.251.0.114:9100,node_disk_read_bytes_total_10.251.0.114:9100,node_disk_read_time_seconds_total_10.251.0.114:9100,...,node_timex_tick_seconds_10.251.0.114:9100,node_uname_info_10.251.0.114:9100,node_vmstat_oom_kill_10.251.0.114:9100,node_vmstat_pgfault_10.251.0.114:9100,node_vmstat_pgmajfault_10.251.0.114:9100,node_vmstat_pgpgin_10.251.0.114:9100,node_vmstat_pgpgout_10.251.0.114:9100,node_vmstat_pswpin_10.251.0.114:9100,node_vmstat_pswpout_10.251.0.114:9100,time
0,6,1620821678,3155029153,0,63548.97,0,27334.180,186083.668,37911559168,1189.212,...,0.01,1,0,626853501,7897,37023016,621356731,0,0,1.626380e+09
1,6,1620821678,3155029535,0,63548.97,0,27334.180,186083.668,37911559168,1189.212,...,0.01,1,0,626853501,7897,37023016,621356731,0,0,1.626380e+09
2,6,1620821678,3155030046,0,63548.99,0,27334.180,186083.668,37911559168,1189.212,...,0.01,1,0,626853501,7897,37023016,621356731,0,0,1.626380e+09
3,6,1620821678,3155030567,0,63549.00,0,27334.180,186083.668,37911559168,1189.212,...,0.01,1,0,626853501,7897,37023016,621356731,0,0,1.626380e+09
4,6,1620821678,3155031313,0,63549.00,0,27334.180,186083.668,37911559168,1189.212,...,0.01,1,0,626853501,7897,37023016,621356731,0,0,1.626380e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,6,1620821678,3163166748,0,65794.22,0,27547.204,186730.896,72843660288,1820.880,...,0.01,1,0,632460820,9563,71136396,626264991,0,0,1.626385e+09
4796,6,1620821678,3163167546,0,65794.23,0,27547.204,186730.896,72843660288,1820.880,...,0.01,1,0,632461449,9563,71136396,626264991,0,0,1.626385e+09
4797,6,1620821678,3163168104,0,65794.24,0,27547.204,186730.896,72843660288,1820.880,...,0.01,1,0,632461449,9563,71136396,626264991,0,0,1.626385e+09
4798,6,1620821678,3163168738,0,65794.25,0,27547.288,186731.328,72843660288,1820.880,...,0.01,1,0,632461451,9563,71136396,626266535,0,0,1.626385e+09


In [14]:
# Add empty target column to clean trace
nodes = nodes
for n in nodes:
    target_cpu = f'target_cpu_{n}'
    target_mem = f'target_mem_{n}'
    target_copy = f"target_copy_{n}"
    target_dummy = f"target_dummy_{n}"
    target_ddot = f"target_ddot_{n}"
    # df_clean[target_cpu] = 0
    # df_clean[target_mem] = 0
    # df_clean[target_copy] = 0
    # df_clean[target_dummy] = 0
    # df_clean[target_ddot] = 0

# print("Clean data columns: ")
# print(df_clean.columns)

# clean_csv = os.path.join(processed_dir, 'df_clean.csv')
# print("Saving clean df to {}".format(clean_csv))
# df_clean.to_csv(clean_csv, index=False)

In [15]:
# Log folder
import glob

list_of_log_dir = glob.glob(os.path.join(anomaly_logs_dir, "logs_*"))


In [16]:
from pygrok import Grok

def setup_grok(pattern):
    grok = Grok(pattern)
    return grok

def parse_logs(file,
               type=None):
    if type == 'copy':
        pattern = '%{NUMBER:unixtime}  anomalies.py:%{NUMBER:linenumber} 	%{LOGLEVEL:loglevel}     %{WORD:status} %{WORD:anomaly_name} with options %{GREEDYDATA:settings} and uuid %{GREEDYDATA:uuid}'
    elif type == 'cpu':
        pattern = '%{NUMBER:unixtime}  anomalies.py:%{NUMBER:linenumber}  	%{LOGLEVEL:loglevel}     %{WORD:status} %{WORD:anomaly_name} with options %{GREEDYDATA:settings} and uuid %{GREEDYDATA:uuid}'
        # pattern = '%{NUMBER:unixtime}  anomalies.py:%{NUMBER:linenumber}%{GREEDYDATA}%{LOGLEVEL:loglevel}     %{WORD:status} %{WORD:anomaly_name} with options %{GREEDYDATA:settiongs} and uuid %{GREEDYDATA:uuid}'
    else:
        pattern = '%{NUMBER:unixtime}  anomalies.py:%{NUMBER:linenumber} 	%{LOGLEVEL:loglevel}     %{WORD:status} %{WORD:anomaly_name} with options %{GREEDYDATA:settings} and uuid %{GREEDYDATA:uuid}'
    grok = setup_grok(pattern)
    with open(file, 'r') as log:
        lines = log.readlines()

    parsed = []
    for line in lines:
        match = grok.match(line)
        if match is None:
            pattern2 = '%{NUMBER:unixtime}  anomalies.py:%{NUMBER:linenumber} 	%{LOGLEVEL:loglevel}     %{WORD:status} %{WORD:anomaly_name} of uuid %{GREEDYDATA:uuid} for'
            grok2 = setup_grok(pattern2)
            match2 = grok2.match(line)
            if match2 is None:
                pass
            else:
                match = match2
        parsed.append(match)
        print(match)
    return parsed

def preflight_labels(parsed_list):
    anomalies = {}
    for e in parsed_list:
        if e is None:
            pass
        else:
            # print(e)
            # print(e['uuid'])
            if e['uuid'] in anomalies.keys():
                pass
            else:
                anomalies[e['uuid']] = {}
            if e['status'] == 'Started':
                anomalies[e['uuid']]['start'] = e['unixtime']
            elif e['status'] == 'Finished':
                anomalies[e['uuid']]['stop'] = e['unixtime']
            elif e['status'] == 'Finised': # todo fix typo for memoryv2
                anomalies[e['uuid']]['stop'] = e['unixtime']
            elif e['status'] == 'Modifier' :
                if 'modifiers' in anomalies[e['uuid']].keys():
                    pass
                else:
                    anomalies[e['uuid']]['modifiers'] = []
                anomalies[e['uuid']]['modifiers'].append(e['unixtime'])
    # print(anomalies)
    return anomalies

def label(df, pre, label, ckcolumn = 'time', target = 'target'):
    df[target] = 0
    for k,v in pre.items():
        target_list = np.where(df[ckcolumn].between(int(float(v['start'])),int(float(v['stop']))), label, df[target])
        df[target] = target_list
    return df

In [17]:
df_labeled = df_anomaly
for logdir in list_of_log_dir:
    print(logdir)
    log_node = logdir.split('-')[-1]
    print(log_node)
    # Log files
    copy_logs = os.path.join(logdir, 'copy-out.log')
    cpu_logs = os.path.join(logdir, 'cpu_overload-out.log')
    ddot_logs = os.path.join(logdir, 'ddot-out.log')
    mem_logs = os.path.join(logdir, 'mem_eater-out.log')
    dummy_logs = os.path.join(logdir, 'dummy-out.log')

    # Parsed logs
    copy_list = parse_logs(copy_logs,type='copy')
    cpu_list = parse_logs(cpu_logs,type='cpu')
    mem_list = parse_logs(mem_logs)
    ddot_list = parse_logs(ddot_logs)
    dummy_list  = parse_logs(dummy_logs)

    pre_cpu = preflight_labels(cpu_list)
    pre_mem = preflight_labels(mem_list)
    pre_copy =preflight_labels(copy_list)
    pre_dummy = preflight_labels(dummy_list)
    pre_ddot = preflight_labels(ddot_list)
    target = f"target_node_{log_node}"

    # Label
    target_cpu = f"target_cpu_{log_node}"
    df_labeled = label(df_labeled, pre_cpu, label = 1, target=target_cpu)
    target_mem = f"target_mem_{log_node}"
    df_labeled = label(df_labeled, pre_mem, label = 1, target=target_mem)
    target_copy = f"target_copy_{log_node}"
    df_labeled = label(df_labeled, pre_copy, label = 1, target=target_copy)
    target_dummy = f"target_dummy_{log_node}"
    df_labeled = label(df_labeled, pre_dummy, label = 1, target=target_dummy)
    target_ddot = f"target_ddot_{log_node}"
    df_labeled = label(df_labeled, pre_ddot, label = 1, target=target_ddot)

/Users/Gabriel/Dropbox/Research/ASPIDE/Datasets/ECI Chaos/Distributed Phase 1/logs/logs_spark-master
master
{'unixtime': '1626380137.369824', 'linenumber': '193', 'loglevel': 'INFO', 'status': 'Started', 'anomaly_name': 'copy', 'settings': '[unit gb, multiplier 1, remove 1, time_out 20]', 'uuid': '52f2a600-7086-4186-8e0b-0a1392c65b9c'}
{'unixtime': '1626380137.370542', 'linenumber': '174', 'loglevel': 'INFO', 'status': 'Started', 'anomaly_name': 'Generating_large_file', 'settings': '[size 1073741824, multiplier 1]', 'uuid': 'gb'}
{'unixtime': '1626380146.575602', 'linenumber': '179', 'loglevel': 'INFO', 'status': 'Finished', 'anomaly_name': 'Generating_large_file', 'settings': '[size 1073741824, multiplier 1]', 'uuid': 'gb'}
{'unixtime': '1626380206.838572', 'linenumber': '213', 'loglevel': 'INFO', 'status': 'Finished', 'anomaly_name': 'copy', 'settings': '[unit gb, multiplier 1, remove 1, time_out 20]', 'uuid': '52f2a600-7086-4186-8e0b-0a1392c65b9c'}
{'unixtime': '1626380207.157193', 

In [18]:
anomaly_labeled = os.path.join(processed_dir, 'df_anomaly.csv')
print("Saving labeled anomaly df to {}". format(anomaly_labeled))
df_labeled.to_csv(anomaly_labeled, index=False)

Saving labeled anomaly df to /Users/Gabriel/Dropbox/Research/ASPIDE/Datasets/ECI Chaos/Distributed Phase 1/processed/df_anomaly.csv


In [19]:
# Check labeled
df_labeled
df_labeled.loc[df_labeled['target_ddot_master'] == 1]


,node_arp_entries_10.251.0.114:9100,node_boot_time_seconds_10.251.0.114:9100,node_context_switches_total_10.251.0.114:9100,node_cpu_guest_seconds_total_10.251.0.114:9100,node_cpu_seconds_total_10.251.0.114:9100,node_disk_io_now_10.251.0.114:9100,node_disk_io_time_seconds_total_10.251.0.114:9100,node_disk_io_time_weighted_seconds_total_10.251.0.114:9100,node_disk_read_bytes_total_10.251.0.114:9100,node_disk_read_time_seconds_total_10.251.0.114:9100,...,node_vmstat_pgpgin_10.251.0.114:9100,node_vmstat_pgpgout_10.251.0.114:9100,node_vmstat_pswpin_10.251.0.114:9100,node_vmstat_pswpout_10.251.0.114:9100,time,target_cpu_master,target_mem_master,target_copy_master,target_dummy_master,target_ddot_master
451,6,1620821678,3155499254,0,63624.28,0,27439.184,186375.672,45976562688,1429.620,...,44899380,623667659,0,0,1.626380e+09,0,0,0,0,1
452,6,1620821678,3155500071,0,63624.89,0,27439.688,186376.572,46031309824,1431.112,...,44953100,623667659,0,0,1.626380e+09,0,0,0,0,1
453,6,1620821678,3155543379,0,63625.76,0,27440.076,186377.244,46073383936,1432.236,...,44992396,623668543,0,0,1.626380e+09,0,0,0,0,1
454,6,1620821678,3155555170,0,63626.54,0,27440.600,186378.252,46113491968,1433.620,...,45032844,623669307,0,0,1.626380e+09,0,0,0,0,1
455,6,1620821678,3155591179,0,63627.34,0,27441.100,186379.200,46165265408,1435.036,...,45083148,623669307,0,0,1.626380e+09,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2448,6,1620821678,3161278466,0,64712.93,0,27530.664,186670.860,66700184576,1739.048,...,65136908,626006555,0,0,1.626382e+09,0,0,0,0,1
2449,6,1620821678,3161279105,0,64713.81,0,27530.664,186670.860,66704247808,1739.084,...,65140876,626006555,0,0,1.626382e+09,0,0,0,0,1
2450,6,1620821678,3161279979,0,64714.81,0,27530.664,186670.860,66707393536,1739.116,...,65143948,626006555,0,0,1.626382e+09,0,0,0,0,1
2451,6,1620821678,3161281088,0,64715.79,0,27530.664,186670.860,66710146048,1739.156,...,65146636,626006659,0,0,1.626382e+09,0,0,0,0,1


In [21]:

df_labeled.loc[df_labeled['target_cpu_master'] == 1]

,node_arp_entries_10.251.0.114:9100,node_boot_time_seconds_10.251.0.114:9100,node_context_switches_total_10.251.0.114:9100,node_cpu_guest_seconds_total_10.251.0.114:9100,node_cpu_seconds_total_10.251.0.114:9100,node_disk_io_now_10.251.0.114:9100,node_disk_io_time_seconds_total_10.251.0.114:9100,node_disk_io_time_weighted_seconds_total_10.251.0.114:9100,node_disk_read_bytes_total_10.251.0.114:9100,node_disk_read_time_seconds_total_10.251.0.114:9100,...,node_vmstat_pgpgin_10.251.0.114:9100,node_vmstat_pgpgout_10.251.0.114:9100,node_vmstat_pswpin_10.251.0.114:9100,node_vmstat_pswpout_10.251.0.114:9100,time,target_cpu_master,target_mem_master,target_copy_master,target_dummy_master,target_ddot_master
501,6,1620821678,3155867027,0,63663.59,0,27461.104,186413.680,48142965760,1489.460,...,47014752,623673535,0,0,1.626380e+09,1,0,0,0,0
502,6,1620821678,3155868124,0,63664.36,0,27461.628,186414.612,48194812928,1490.976,...,47065128,623673623,0,0,1.626380e+09,1,0,0,0,0
503,6,1620821678,3155868791,0,63665.06,0,27462.144,186415.584,48243440640,1492.336,...,47113128,623673623,0,0,1.626380e+09,1,0,0,0,0
504,6,1620821678,3155869540,0,63665.53,0,27462.680,186416.556,48292461568,1493.936,...,47161384,623673623,0,0,1.626380e+09,1,0,0,0,0
505,6,1620821678,3155870323,0,63666.24,0,27463.140,186417.524,48340433920,1495.316,...,47207720,623673623,0,0,1.626380e+09,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539,6,1620821678,3161521762,0,64768.32,0,27531.320,186672.832,67006131200,1742.376,...,65435684,626018275,0,0,1.626382e+09,1,0,0,0,0
2540,6,1620821678,3161522607,0,64769.28,0,27531.320,186672.832,67009801216,1742.416,...,65439268,626018275,0,0,1.626382e+09,1,0,0,0,0
2541,6,1620821678,3161523678,0,64770.22,0,27531.320,186672.832,67012422656,1742.424,...,65441828,626018347,0,0,1.626382e+09,1,0,0,0,0
2542,6,1620821678,3161524773,0,64771.18,0,27531.320,186672.832,67015175168,1742.448,...,65444516,626018347,0,0,1.626382e+09,1,0,0,0,0
